In [1]:
# remove edges with low counts/occurence, or no good mapping, or not useful

In [1]:
import os
import pickle
%pylab
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict
from tqdm import tqdm

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
edges = pd.read_csv('edges_sanitized.csv')

In [3]:
nodes = pd.read_csv("nodes_blm.csv", index_col=0)
nodes.head()

,ID,label,umls_type,umls_type_label,blm_type
1,C1290952,Taking medication,T056,Daily or Recreational Activity,activity_and_behavior
3,C0085092,Parenting behavior,T054,Social Behavior,activity_and_behavior
4,C1096771,Murderer,T055,Individual Behavior,activity_and_behavior
6,C0006875,Cannibalism,T054,Social Behavior,activity_and_behavior
7,C0871454,Study Habits,T055,Individual Behavior,activity_and_behavior


In [4]:
print(len(nodes))
print(len(edges))

180151
20620112


In [5]:
# remove edges with no nodes
print(len(edges))
edges = edges[edges.START_ID.isin(nodes.ID) & edges.END_ID.isin(nodes.ID)]
print(len(edges))

20620112
14145903


In [6]:
abv = pd.read_csv("abv.csv")
name_abv = dict(zip(abv.full_name, abv.abv))
abv_name = dict(zip(abv.abv, abv.full_name))

In [7]:
edges.head()

,START_ID,END_ID,TYPE,pmids,n_pmids,NEG,DOMAIN,PRED,RANGE
0,C1273870,C0282623,ABatAB,17170614,1,False,AB,at,AB
7,C1273870,C0021294,ABatDO,22564132,1,False,AB,at,DO
8,C1273870,C0581876,ABatDO,20937431,1,False,AB,at,DO
413,C1517631,C1321301,AatA,17302772,1,False,A,at,A
414,C1167519,C0007634,AatA,19232422,1,False,A,at,A


In [8]:
vc = edges.PRED.value_counts()
vc.index = vc.index.map(lambda x:abv_name.get(x))
vc

LOCATION_OF         1832484
INTERACTS_WITH      1613110
COEXISTS_WITH       1565429
AFFECTS             1334197
PART_OF             1170120
STIMULATES           978262
INHIBITS             803144
ASSOCIATED_WITH      723481
CAUSES               691203
AUGMENTS             612280
TREATS               595909
PRODUCES             492690
DISRUPTS             444683
compared_with        309671
ISA                  241195
PREDISPOSES          208151
PREVENTS             114242
higher_than           97908
CONVERTS_TO           55314
PROCESS_OF            54944
MANIFESTATION_OF      36117
COMPLICATES           33029
USES                  30890
PRECEDES              24445
lower_than            19562
same_as               19549
OCCURS_IN             15969
ADMINISTERED_TO       14707
different_from         5937
DIAGNOSES              5409
different_than         1434
METHOD_OF               438
Name: PRED, dtype: int64

In [9]:
def drop_edges(df, drop_edges):
    idx = df.query('PRED in @drop_edges').index
    df.drop(idx, inplace=True)

In [10]:
# remove edges we don't need
print(len(edges))
remove_edges = ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 'same_as',
               'OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES',
               'AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']
drop_edges(edges, list(map(name_abv.get, remove_edges)))
print(len(edges))

14145903
12924176


In [11]:
edge_type_counts = edges.TYPE.value_counts()
print(edge_type_counts.head())
print(edge_type_counts.tail())

CDiwCD    1003118
AloCD      849621
DOcwDO     591441
CDcwCD     543724
CDstCD     525319
Name: TYPE, dtype: int64
GiPS      1
OBinA     1
ABpdDO    1
ABstA     1
DOpoPH    1
Name: TYPE, dtype: int64


In [12]:
# somewhat arbitrarily remove type->pred->type edges with less than 1000 instances
print(len(edge_type_counts))
idx = edge_type_counts[edge_type_counts>1000]
print("keeping {} edges out of {}".format(len(idx), len(edge_type_counts)))
orig_len = len(edges)
edges = edges[edges.TYPE.isin(idx.index)]
print("keeping {} out of {} rows".format(len(edges), orig_len))

578
keeping 141 edges out of 578
keeping 12880620 out of 12924176 rows


In [13]:
edge_type_counts = edges.TYPE.value_counts()

In [14]:
edge_type_counts[edge_type_counts.index.str.contains("cw")]

DOcwDO    591441
CDcwCD    543724
CDcwG     253819
GcwG      112263
PScwPS     33957
PScwDO      9901
GcwCD       8453
PHcwDO      2327
GcwA        1756
OBcwCD      1371
ABcwDO      1212
Name: TYPE, dtype: int64

In [15]:
# remove nodes with no edges
print(len(nodes))
nodes = nodes[nodes.ID.isin(set(list(edges.START_ID) + list(edges.END_ID)))]
print(len(nodes))

180151
165577


In [16]:
vc = edges.PRED.value_counts()
vc.index = vc.index.map(lambda x:abv_name.get(x))
vc

LOCATION_OF         1828906
INTERACTS_WITH      1610497
COEXISTS_WITH       1560224
AFFECTS             1330905
PART_OF             1168114
STIMULATES           975601
INHIBITS             800502
ASSOCIATED_WITH      719086
CAUSES               689445
TREATS               591775
PRODUCES             491415
DISRUPTS             441921
ISA                  239894
PREDISPOSES          206403
PREVENTS             112275
CONVERTS_TO           55158
MANIFESTATION_OF      35088
PRECEDES              23411
Name: PRED, dtype: int64

In [17]:
edges.to_csv('edges_filtered.csv', index=False)
nodes.to_csv("nodes_filtered.csv", index=False)